In [2]:
import numpy as np    
import matplotlib.pyplot as plt
import pandas as pd 

In [4]:
## import dataset
dataset = pd.read_csv("Mall_Customers.csv")
dataset

In [8]:
x = dataset.iloc[:, [3,4]].values

,Annual Income (k$),Spending Score (1-100)
0,15,39
1,15,81
2,16,6
3,16,77
4,17,40
...,...,...
195,120,79
196,126,28
197,126,74
198,137,18
